## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hofn,IS,64.2539,-15.2082,44.78,86,11,0.45,few clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,75,6.91,broken clouds
2,2,Bulawayo,ZW,-20.1500,28.5833,59.56,89,78,5.08,broken clouds
3,3,Padang,ID,-0.9492,100.3543,75.25,85,100,3.83,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,58.17,95,100,1.01,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 50
What is your desired maximum temperature for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bulawayo,ZW,-20.1500,28.5833,59.56,89,78,5.08,broken clouds
3,3,Padang,ID,-0.9492,100.3543,75.25,85,100,3.83,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,58.17,95,100,1.01,moderate rain
5,5,Hamilton,US,39.1834,-84.5333,76.42,51,40,6.91,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,71.29,88,100,7.81,moderate rain
7,7,Albany,US,42.6001,-73.9662,55.81,48,10,8.88,clear sky
9,9,Butaritari,KI,3.0707,172.7902,82.09,74,20,19.39,few clouds
11,11,Bredasdorp,ZA,-34.5322,20.0403,56.89,61,81,6.76,broken clouds
12,12,Lapuyan,PH,7.6379,123.1944,79.09,86,52,2.71,broken clouds
13,13,Ailigandi,PA,9.2333,-78.0167,80.65,75,19,1.54,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bulawayo,ZW,-20.1500,28.5833,59.56,89,78,5.08,broken clouds
3,3,Padang,ID,-0.9492,100.3543,75.25,85,100,3.83,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,58.17,95,100,1.01,moderate rain
5,5,Hamilton,US,39.1834,-84.5333,76.42,51,40,6.91,scattered clouds
6,6,Rikitea,PF,-23.1203,-134.9692,71.29,88,100,7.81,moderate rain
...,...,...,...,...,...,...,...,...,...,...
695,695,Saint-Pierre,RE,-21.3393,55.4781,74.88,79,0,28.77,clear sky
696,696,Debre Tabor,ET,11.8500,38.0167,57.16,65,100,2.46,light rain
697,697,Harper,LR,4.3750,-7.7169,78.64,83,44,2.77,scattered clouds
698,698,Poum,NC,-20.2333,164.0167,77.72,69,57,7.20,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bulawayo,ZW,59.56,broken clouds,-20.1500,28.5833,
3,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,
4,Port Alfred,ZA,58.17,moderate rain,-33.5906,26.8910,
5,Hamilton,US,76.42,scattered clouds,39.1834,-84.5333,
6,Rikitea,PF,71.29,moderate rain,-23.1203,-134.9692,
7,Albany,US,55.81,clear sky,42.6001,-73.9662,
9,Butaritari,KI,82.09,few clouds,3.0707,172.7902,
11,Bredasdorp,ZA,56.89,broken clouds,-34.5322,20.0403,
12,Lapuyan,PH,79.09,broken clouds,7.6379,123.1944,
13,Ailigandi,PA,80.65,few clouds,9.2333,-78.0167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bulawayo,ZW,59.56,broken clouds,-20.1500,28.5833,4 On Housman Backpackers
3,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,Mercure Padang
4,Port Alfred,ZA,58.17,moderate rain,-33.5906,26.8910,The Halyards Hotel
5,Hamilton,US,76.42,scattered clouds,39.1834,-84.5333,Six Acres Bed & Breakfast
6,Rikitea,PF,71.29,moderate rain,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))